# Homework 3 (Due Oct. 11)

## 1. The occasionally dishonest casino

Imagine that you are employed by the Montana state gaming board, and you have received an anonymous tip suggesting that the dealer in the dice games that frequently occur at the Oxford is cheating.  In particular, the dealer is surreptitiously switching from a fair die to a loaded one (and back) with a certain probability.  In this game, the ante is a doller.  you choose a number, then the dealer then rolls the die and if it matches you get back 3 times your ante.  Otherwise, you lose your ante.  

You examine security camera footage from which you extracted the following dataset, with *states* describing whether the die was loaded, and rolls being the outcome of each die roll.

In [ ]:
import pickle
import matplotlib.pyplot as plt

states,rolls = pickle.load(open('oxford.p','rb'))
plt.plot(states)
plt.show()

### 1a. Training a HMM (20pts)
Assume that the dealer's choice of die follows a Markov model:
$$
P(S_{t+1}=s_k|S_t=s_j) = A_{jk}
$$
and the die roll follows a categorical distribution conditioned on which die was being used, e.g.
$$
P(R_t=r|S_t=s_j) = E_{jr}.
$$
Train your model of the dealer's ruse by using your observations to find the maximum likelihood estimators for the transition matrix $A$ and the emission probabilities $E$.

In [ ]:
import numpy as np
m = len(states)

A = np.zeros((2,2))
E = np.zeros((2,6))
#! Compute the parameters of the transition and emission matrices.

### 1b. Determining when the house is cheating. (20pts)
Now that you know the cheating dealer's strategy, you decide that you want to make a little bit of money off the house.  Play 1000 games of dice with the dealer, using the forward algorithm to determine when the dealer is cheating.  Recall that you now don't have access to the dealer's state, only the die rolls!  Assume that the dealer starts using the fair die.  Report the probability that the dealer is using the loaded die at every roll.

In [ ]:
def likelihood(roll,state):
    return E[state,roll]

def prediction(state):
    return np.dot(state,A)

possible_states = [0,1]
possible_rolls = [0,1,2,3,4,5]

alpha = [np.array([1.0,0.0])]
current_state = 0
current_roll = np.random.choice(possible_rolls,p=E[0])

states = [current_state]
rolls = [current_roll]

for g in range(1000):
        
    ### This is the dealer's internal state, you can't use this information!
    current_state = np.random.choice(possible_states,p=A[current_state])
    states.append(current_state)
    ###
    
    ### This is the dealer's emission, you *can* use this information
    current_roll = np.random.choice(possible_rolls,p=E[current_state])
    rolls.append(current_roll)
    ###
    
    #! Implement the recursive forward algorithm to determine the probability 
    #! of being in the fair or loaded state for all rolls

### 1C (\*). Beating the house (10pts)
Describe a strategy for beating the house given all the information determined from the above two problems.  Show that your strategy will, in general, make money after 1000 rolls, either by mathematical proof, or by direct simulation. 

## 2. The Kalman Filter

### 2a.  Tracking in 2D.  (30pts)
Imagine that we are tracking an airplane using a global positioning unit attached to the hull.  At a fixed interval, the unit returns a noisy observation $\mathbf{Z}_t = H \mathbf{X}_t + \mathbf{g}_t$, where $t$ is the current time, $\mathbf{X}=[x,u,y,v]$ is the true position and velocity of the aircraft (in the horizontal plane),
$$
H = \begin{bmatrix} 1 & 0 & 0 & 0 \\
                    0 & 0 & 1 & 0 \end{bmatrix},
$$
and $\mathbf{g}_t$ is Gaussian white noise distributed according to $\mathbf{g}_t \sim \mathcal{N}(0,\mathbf{R}_t)$, where $\mathbf{R}_t$ is the observation covariance.  

To obtain a better estimate of the position of the aircraft, we can apply a Kalman filter.  First, we must assume that the airplane moves with constant velocity, subject to buffeting by winds that are independent in each dimension.  Thus, we have that 
$$
\mathbf{X}_{t+1} = A \mathbf{X}_t + \mathbf{w}_t
$$

where 
$$
A = \begin{bmatrix} 1 & \Delta t & 0 & 0 \\
                    0 & 1 & 0 & 0  \\
                    0 & 0 & 1 & \Delta t \\
                    0 & 0 & 0 & 1 \end{bmatrix}
$$
is a transition matrix and $\mathbf{w}_t \sim \mathcal{N}(0,Q)$ is Gaussian distributed noise drawn from a covariance matrix Q.  In our case, 
$$
Q = a^2 \begin{bmatrix} 1/4 \Delta t^4 & 1/2\Delta t^3 & 0 &         0 \\
                        1/2 \Delta t^3 & \Delta t^2    & 0 &         0 \\
                        0 & 0 & 1/4 \Delta t^4 & 1/2\Delta t^3 \\
                        0 & 0 & 1/2 \Delta t^3 & \Delta t^2    \end{bmatrix},                
$$
where $a$ is the standard deviation of accelerations due to wind.  This covariance matrix implies that wind forces are independent in the horizontal dimensions.  

We seek an estimate of $\mathbf{X}_{post}$, as well as an estimate of its covariance $P_{post}$, which will optimally combine our knowledge of how the plane moves with our observations of its position.  Apply the Kalman filter to the aircraft motion and resulting observations simulated below.  After applying the filter, generate a plot of the true aircraft positions, the observations of the aircraft, and the position estimates generated by filtering.

In [ ]:
import numpy as np

# Constants
a = 0.1   # Acceleration due to wind
dt = 0.1  # Interval for measurements

# Transition matrix
A = np.array([[1.0,dt,0.0,0.0],
              [0.0,1.0,0.0,0],
              [0.0,0.0,1.0,dt],
              [0.0,0.0,0,1.0]])

# Process noise covariance
Q = a**2*np.array([[1/4.*dt**4,1/2.*dt**3,         0,         0],
                   [1/2.*dt**3,     dt**2,         0,         0],
                   [         0,         0,1/4.*dt**4,1/2.*dt**3],
                   [         0,         0,1/2.*dt**3,     dt**2]])

# Observation matrix
H = np.array([[1,0,0,0],
              [0,0,1,0]])

# Observation covariance
R = np.array([[0.05**2,0],
              [0,0.05**2]])

### These are useful for sampling from normal distributions, but you
### don't need to worry about them.  
Q_cholesky = np.linalg.cholesky(Q)
R_cholesky = np.linalg.cholesky(R)

### The true state of the aircraft
### You do not have access to this information!
###
X_true = [np.array([-1,0.1,-1,0.1])]
###
###
###

# Initial guess for aircraft position and velocity
X_initial = np.array([0,0,0,0])

# Covariance matrix for this initial guess of position and velocity 
P_initial = np.eye(4)*np.diag(np.array([1,1,1,1]))

# Lists for holding estimated states
X = [X_initial]
P = [P_initial]
Z = []


t=0
while t<t_end:
    
    ### Simulate the motion of the aircraft
    ### You do not have access to this information!
    ### Don't change!
    X_0 = X_true[-1]                                            
    X_1 = np.dot(A,X_0) + np.dot(Q_cholesky,np.random.randn(4))  # Move the plane forward in time
    X_true.append(X_1)
    Z_1 = np.dot(H,X_1) + np.dot(R_cholesky,np.random.randn(2))  # Take a noisy measurement
    Z.append(Z_1)
    ###
    ###
    ###
    
    X_previous = X[-1] # Best state mean estimate from last time step
    P_previous = P[-1] # state covariance from last time step
    Z_current = Z[-1]  # Current observation
    
    # Prediction step
    #! Write code to predict
    X_prior = None #! a) where the airplane will be according to the motion model
    P_prior = None #! b) b) the covariance of that estimate
   
    # Update step
    #! Implement the formulae for the update step of the Kalman filter
    #!
    #!
    #!
    X_post = None #! The posterior state mean estimate 
    P_post = None #! The posterior covariance estimate
    
    X.append(X_post)
    P.append(P_post)
    
    t+=dt
    
#! Plotting goes here.

### 2b (\*) Extended Kalman Filter. (20pts)
The Kalman filter can only apply to observation models that are linear.  However, this is a restrictive assumption.  Consider the case where instead of tracking the plane with a GPS, we are tracking it with a radar, where the radar provides an estimate of range ($R$) and angle ($\theta$).  The process of taking an observation is non-linear, which is to say that 
$$
\mathbf{Z}_t = h(\mathbf{X}_t) + \mathbf{g}_t
$$
where $h(\mathbf{X}_t)$ is the transformation from Cartesian to Polar coordinates
$$ h(\mathbf{X}_t) = \left[\sqrt{x_t^2 + y_t^2},\mathrm{arctan2}(y_t,x_t)\right]. $$

An extended Kalman filter works by linearizing the observation process, which is to say that every time we see the product $H\mathbf{X_t}$ in the Kalman filter algorithm, we replace it with $h(\mathbf{X_t})$, and everytime we see $H$ on its own, we replace it with the **Jacobian** matrix of $h(\mathbf{X_t})$, which is given by
$$
H_{jac} = \begin{bmatrix} \frac{\partial r_t}{\partial x_t} & 0 & \frac{\partial r_t}{\partial y_t} & 0 \\
                          \frac{\partial \theta_t}{\partial x_t} & 0 & \frac{\partial \theta_t}{\partial y_t} & 0
                          \end{bmatrix}
$$
Compute this Jacobian matrix and alter your Kalman Filter code to implement the Extended Kalman Filter.  Note that the code skeleton produces observations that are now in terms of $r$ and $\theta$.  Note also that the observational covariance $R$ has been altered to account for the different magnitude of errors in these values. 

In [ ]:
import numpy as np

# Constants
a = 0.1   # Acceleration due to wind
dt = 0.1  # Interval for measurements

# Transition matrix
A = np.array([[1.0,dt,0.0,0.0],
              [0.0,1.0,0.0,0],
              [0.0,0.0,1.0,dt],
              [0.0,0.0,0,1.0]])

# Process noise covariance
Q = a**2*np.array([[1/4.*dt**4,1/2.*dt**3,         0,         0],
                   [1/2.*dt**3,     dt**2,         0,         0],
                   [         0,         0,1/4.*dt**4,1/2.*dt**3],
                   [         0,         0,1/2.*dt**3,     dt**2]])


# Observation covariance
R = np.array([[0.1**2,0],
              [0,0.01**2]])

# Observation function
def h(X):
    R = np.sqrt(X[0]**2 + X[2]**2)
    theta = np.arctan2(X[2],X[0])
    return np.array([R,theta])

# Jacobian of observation function
def H_jac(X):
    #! Generate a function which returns the Jacobian of h
    pass

### These are useful for sampling from normal distributions, but you
### don't need to worry about them.  
Q_cholesky = np.linalg.cholesky(Q)
R_cholesky = np.linalg.cholesky(R)

### The true state of the aircraft
### You do not have access to this information!
###
X_true = [np.array([-1,0.1,-1,0.1])]
###
###
###

# Initial guess for aircraft position and velocity
X_initial = np.array([-1.0,0.1,-1.0,0.1])

# Covariance matrix for this initial guess of position and velocity 
P_initial = np.eye(4)*1#np.diag(np.array([1,1,1,1]))

# Lists for holding estimated states
X = [X_initial]
P = [P_initial]
Z = []

print(np.dot(H_jac(X_initial),X_initial))
print(h(X_initial))

t=0
t_end = 10.0
while t<t_end:
    
    ### Simulate the motion of the aircraft
    ### You do not have access to this information!
    ### Don't change!
    X_0 = X_true[-1]                                            
    X_1 = np.dot(A,X_0) + np.dot(Q_cholesky,np.random.randn(4))  # Move the plane forward in time
    X_true.append(X_1)
    Z_1 = h(X_1) + np.dot(R_cholesky,np.random.randn(2))  # Take a noisy measurement
    Z.append(Z_1)
    ###
    ###
    ###
    
    X_previous = X[-1] # Best state estimate from last time step
    P_previous = P[-1] # state covariance from last time step
    Z_current = Z[-1]  # Current observation
    
    H = H_jac(X_previous)
    
    # Prediction step
    #! Write code to predict
    X_prior = None #! a) where the airplane will be according to the motion model
    P_prior = None #! b) b) the covariance of that estimate
   
    # Update step
    #! Implement the formulae for the update step of the Kalman filter
    #!
    #!
    #!
    X_post = None #! The posterior state mean estimate 
    P_post = None #! The posterior covariance estimate
    
    X_post = X_prior + np.dot(K,y_tilde)
    P_post = np.dot(np.eye(4) - np.dot(K,H),P_prior)
    
    X.append(X_post)
    P.append(P_post)
    
    t+=dt
